# Imports

Learning about classification tasks.

I will be using the MNIST dataset. It consists of images of handwritten digits (0-9).

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [6]:
# Plot style

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# MNIST dataset

## Setup

In [5]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [7]:
IMAGE_PATH = Path("../images/classification")
IMAGE_PATH.mkdir(parents=True, exist_ok=True)